In [1]:
from sfcn import SFCN
import numpy as np 

2021-12-06 12:01:12.655111: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
model = SFCN(
    input_dim=[160, 192, 160, 1], 
    output_dim=1,
    conv_num_filters=[32, 64, 128, 256, 256, 64], 
    conv_kernel_sizes=[3, 3, 3, 3, 3, 1], 
    conv_strides=[1, 1, 1, 1, 1, 1],
    conv_padding=['same', 'same', 'same', 'same', 'same', 'valid'],
    maxpool_size=[2, 2, 2, 2, 2],
    batch_norm=False,
    dropout=False,
    softmax=False,
    gpu_num=2
    )

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


2021-12-06 12:01:17.946150: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-12-06 12:01:17.949220: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-12-06 12:01:18.576801: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:04:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2021-12-06 12:01:18.579937: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:06:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2021-12-06 12:01:18.583087: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 2 with properties: 
pciBusID: 0000:07:0

In [3]:
model.compile(learning_rate=1e-6)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 160, 192, 160, 1) 0         
_________________________________________________________________
conv_0 (Conv3D)              (None, 160, 192, 160, 32) 896       
_________________________________________________________________
maxpool_0 (MaxPooling3D)     (None, 80, 96, 80, 32)    0         
_________________________________________________________________
activation_0 (Activation)    (None, 80, 96, 80, 32)    0         
_________________________________________________________________
conv_1 (Conv3D)              (None, 80, 96, 80, 64)    55360     
_________________________________________________________________
maxpool_1 (MaxPooling3D)     (None, 40, 48, 40, 64)    0         
_________________________________________________________________
activation_1 (Activation)    (None, 40, 48, 40, 64)    0     

In [4]:
data_size = 200

In [5]:
X = np.random.normal(loc=5, scale=5, size=(data_size, 160, 192, 160, 1))

In [6]:
# X = np.random.no(0, 3, size=(data_size, 160, 192, 160, 1))

In [7]:
y = np.mean(X, axis=(1,2,3,4))

In [9]:
model.train(X, y, 4, 300)

2021-12-06 12:03:26.341769: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:656] In AUTO-mode, and switching to DATA-based sharding, instead of FILE-based sharding as we cannot find appropriate reader dataset op(s) to shard. Error: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_2715"
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 4
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset. You can do this by creating a new `tf.data.Options()` object then setting `options.experimental_distribute.auto

Epoch 1/300
INFO:tensorflow:batch_all_reduce: 14 all-reduces with algorithm = nccl, num_packs = 1
30/30 [==============================] - ETA: 0s - loss: 17.2440 - mae: 4.1349

2021-12-06 12:04:41.385381: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:656] In AUTO-mode, and switching to DATA-based sharding, instead of FILE-based sharding as we cannot find appropriate reader dataset op(s) to shard. Error: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_3760"
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 4
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset. You can do this by creating a new `tf.data.Options()` object then setting `options.experimental_distribute.auto

30/30 [==============================] - 78s 508ms/step - loss: 17.2440 - mae: 4.1349 - val_loss: 12.0230 - val_mae: 3.4674
Epoch 2/300
30/30 [==============================] - 11s 378ms/step - loss: 8.4541 - mae: 2.8874 - val_loss: 5.2191 - val_mae: 2.2845
Epoch 3/300
30/30 [==============================] - 11s 380ms/step - loss: 3.1698 - mae: 1.7524 - val_loss: 1.4314 - val_mae: 1.1964
Epoch 4/300
30/30 [==============================] - 12s 392ms/step - loss: 0.6238 - mae: 0.7498 - val_loss: 0.1237 - val_mae: 0.3517
Epoch 5/300
30/30 [==============================] - 12s 388ms/step - loss: 0.0327 - mae: 0.1514 - val_loss: 3.2705e-04 - val_mae: 0.0177
Epoch 6/300
30/30 [==============================] - 11s 369ms/step - loss: 1.7727e-04 - mae: 0.0121 - val_loss: 2.0680e-04 - val_mae: 0.0139
Epoch 7/300
30/30 [==============================] - 11s 357ms/step - loss: 7.0272e-05 - mae: 0.0072 - val_loss: 1.4369e-05 - val_mae: 0.0032
Epoch 8/300
30/30 [==============================] -

In [9]:
x_test = np.random.normal(loc=8, scale=3, size=(1, 160, 192, 160, 1))

In [10]:
model.predict(x_test)

2021-12-06 11:32:43.345462: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:656] In AUTO-mode, and switching to DATA-based sharding, instead of FILE-based sharding as we cannot find appropriate reader dataset op(s) to shard. Error: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_12409"
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset. You can do this by creating a new `tf.data.Options()` object then setting `options.experimental_distribute.au

array([[3.4749513]], dtype=float32)

In [11]:
#y_test = np.mean(x_test, axis=(1,2,3,4))
#print(y_test)